In [1]:
from linkedin import LinkedIn

# selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# other
import pandas as pd
import time
import os

# credentials
email = 'joseph.barbierdarnal@gmail.com'
with open('../credentials.txt', 'r') as file:
    password = file.read()

In [2]:
scraper = LinkedIn()
scraper.load_page(
    "https://www.linkedin.com/",
    try_quitting_first=False
)
scraper.sleep(5)
scraper.accept_cookies()
scraper.login(email, password)

In [3]:
scraper.load_page(
    "https://www.linkedin.com/jobs/collections/recommended/",
    try_quitting_first=False
)

In [4]:

#scraper.close_message()
scraper.enter_keywords('data science')
scraper.enter_location('European Economic Area')
scraper.press_enter()
scraper.scroll_to_bottom()
scraper.scrap_jobs()

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=120.0.6099.216)
Stacktrace:
0   chromedriver                        0x0000000102b704dc chromedriver + 4162780
1   chromedriver                        0x0000000102b68664 chromedriver + 4130404
2   chromedriver                        0x00000001027bfbc0 chromedriver + 293824
3   chromedriver                        0x00000001027c5234 chromedriver + 315956
4   chromedriver                        0x00000001027c74d0 chromedriver + 324816
5   chromedriver                        0x000000010284646c chromedriver + 844908
6   chromedriver                        0x0000000102845ab0 chromedriver + 842416
7   chromedriver                        0x00000001027f96bc chromedriver + 530108
8   chromedriver                        0x00000001027fa930 chromedriver + 534832
9   chromedriver                        0x0000000102b35e08 chromedriver + 3923464
10  chromedriver                        0x000

In [8]:
pd.read_csv('../data/job_df.csv')

,Job ID,Title,Company,Location,Description,Date
0,3800692625,Data Scientist confirmé,Mobiskill - Recrutement Tech,"Paris, Île-de-France, France (Hybrid)",Data Scientist confirmé\nMobiskill - Recruteme...,6 days ago
1,3800940696,Data Analyst,Hays,"Vienna, Austria (Hybrid)","Data Analyst\nHays\nVienna, Austria (Hybrid)\n...",6 days ago
2,3786715723,(Associate) Evaluation Data Scientist - based ...,European Investment Bank (EIB),"Ljubljana, Ljubljana, Slovenia (On-site)",(Associate) Evaluation Data Scientist - based ...,6 days ago
3,3800906738,Machine Learning Engineer,Albert Bow,"Malmö, Skåne County, Sweden (On-site)","Machine Learning Engineer\nAlbert Bow\nMalmö, ...",6 days ago
4,3780058871,Data scientist graduate,PFA,"København, Capital Region, Denmark (On-site)","Data scientist graduate\nPFA\nKøbenhavn, Capit...",6 days ago
...,...,...,...,...,...,...
127,3798403288,Consultant Luftfracht / Airfreight (m/w/d),CargoSoft GmbH e-logistic solutions,"Bremen, Bremen, Germany (Hybrid)",Consultant Luftfracht / Airfreight (m/w/d)\nCa...,6 days ago
128,3798365063,PLC Programmer,LEMATEC s.r.o.,"Brno, South Moravia, Czechia (Hybrid)","PLC Programmer\nLEMATEC s.r.o.\nBrno, South Mo...",6 days ago
129,3798358352,Data Scientist,Wargaming,"Vilnius, Vilniaus, Lithuania (Hybrid)","Data Scientist\nWargaming\nVilnius, Vilniaus, ...",6 days ago
130,3797371211,Werkstudent:in Personalmarketing,cronos Unternehmensberatung GmbH,"Münster, North Rhine-Westphalia, Germany (Hybrid)",Werkstudent:in Personalmarketing\ncronos Unter...,6 days ago
